# Visualización de tratamientos de radioterapia
------

Para la prescripción, planificación y valoración de un tratamiento de radioterapia es necesario conocer la anatomía del paciente, la descripción de los haces empleados y la distribución de dosis generada.

## Estudio de CT

Los datos anatómicos para la *simulación* del tratamiento del paciente se adquieren mediante un estudio de tomografía computerizada *CT* con el paciente inmobilizaddo en una postura que sea reproducible a lo largo del tratamiento.

Los volúmenes relevantes para el tratamiento se escanean mediante cortes axiales que permiten realizar una reconstrucción tridimensional del paciente.

En este cuaderno se puede visualizar el estudio de radioterapia de una paciente de carcinoma de mama.

Comenzamos yendo al directorio que contiene los datos del estudio de la paciente e importando los módulos necesarios. 

Utilizaremos una librería específica para la visualización y evaluación de planes de radioterapia: `scikit-rt`

In [ ]:
%cd ..

In [ ]:
# Librerias específicas de radioterapia
from skrt import Image, StructureSet, Patient
from dicompylercore import dicomparser, dvh, dvhcalc

# La libreria general DICOM
import pydicom as dcm
# Manejo de listas de archivos
from glob import glob
# Otras librerias para facilitar la visualización
from matplotlib import pyplot as plt
import ipywidgets as wdg
%matplotlib widget

Leemos el estudio CT

In [ ]:
im = Image('patients/TestMamaDer001/20230116_095704/CT/20230116_095704')

En la figura generada en la siguiente celda es posible ajustando las barras de desplazamiento navegar por la anatomía de la paciente.

Se puede apreciar que la paciente está situada con los brazos levantados para alejarlos de la región de tratamiento. Esto se consigue mediante unos inmovilizadores en los que la paciente apoya los brazos y los antebrazos en posiciones ajustables y reproducibles que faciliten el confort de la paciente. También es visible que la paciente tiene apoyada la espalda sobre un soporte completamente plano.

En la posición de partida se pueden apreciar en el corte axial la posición de la cicatriz de la operación indicada mediante una marca radioopaca. El resto de puntos brillantes en el estudio son marcas externas relativas al posicionamiento del paciente que se utilizan para reproducir su posición y establecer un marco de coordenadas al que referir el tratamiento.

In [ ]:
@wdg.interact(x=(-200, 200), y=(-400, 0), z=(-50, 280)) 
def view3D(x, y, z):
    fig, axd = plt.subplot_mosaic([['ax', 'ax'], ['sg', 'cr']], figsize=[10,10])
    im.plot('x-y', ax=axd['ax'], pos=z, show=False, title='Vista axial')
    im.plot('y-z', ax=axd['sg'], pos=x, show=False, title='Vista sagital')
    im.plot('x-z', ax=axd['cr'], pos=y, title='Vista coronal')

En la discusión que sigue solo nos referiremos a los planos axiales recordando que otras vistas son posible modificando el primer argumento de las funciones de visualización. Por defecto este argumento asume proyección 'x-y', planos axiales.

In [ ]:
@wdg.interact(z=(-50, 280))
def axim(z):
    # Selcción de orientación de los planos  
    ## 'x-y' Axial
    ## 'x-z' Coronal
    ## 'y-z' Sagital  
    im.plot('x-y', pos = z, figsize=8) 

El estudio de CT se adquirió con cortes cada 3 mm. La visualización muestra el plano más cercano al valor indicado por el parámetro `pos`.  

## Estructuras

Los volumenes relevantes para la planificación, sean bien objetivos o bien regiones de interés, se tienen que delimitar sobre el estudio CT de simulación.

Las imágenes se tienen que segmentar mediante procesos manuales o semiautomáticos. Estos últimos utilizan procedimientos de base de conocimiento que se ha tenido que establecer previamente mediante procesos manuales.

Leemos el conjunto de estructuras presentes en el estudio

In [ ]:
ss = StructureSet('patients/TestMamaDer001/20230116_095704/RTSTRUCT/CT/20230116_095704/RS.1.2.246.352.205.5566667512400723974.375329800850442140.dcm')

Los conjuntos de estructuras se visualizan asociándolos a la imagen a partir de la que han sido generados

In [ ]:
im.add_structure_set(ss)

In [ ]:
im.plot(rois='all', pos=115, legend=True, legend_loc='lower right', figsize=8)

Para la planificación son relevantes de estas estructuras el contorno exterior (BODY) y la modelo de la mesa (COUCH). El BODY restringe a los puntos en su interior el cálculo de interacciones. Cualquier voxel externo al BODY se considera vacío en términos físicos a excepción de los volúmenes COUCH. Sus densidades se fuerzan a las correspondientes a los materiales con los que están hechos los elementos de soporte del acelerador.

Por simplificar del resto de estructuras solo consideraremos los PTVs, volúmenes objetivos de la prescripción, y como región de interés el pulmón ipsilateral, en este caso el pulmón derecho. Para entender que se trata del pulmón derecho del paciente consideresé que la paciente está situada como si sus pies estuvieran saliendo de la pantalla. En el estudio CT esta orientación se indentifica como *head first*

## Paciente

Desde el punto de vista de la radioterapia una manera más conveniente de almacenar los datos de tratamiento es mediante una estructura en la que a un estudio de CT se le asocian una o varias conjuntos de estructuras, uno o varios planes de tratamiento y una o varias distribuciones de dosis.

Las distribuciones de dosis se pueden visualizar de forma independiente aunque técnicamente tienen que estar asociadas al plan que permite generarlas.

Planes y distribuciones de dosis van asociadas a los conjuntos de estructuras.

Las imágenes de un paciente realizadas en un momento concreto y los conjuntos de estructuras, planes y distribuciones de dosis asociadas forman un **estudio**.

`scikit-rt` requiere una estrctura concreta de directorios para almacener todos estos datos y poder identificar al paciente como a los estudios que contiene.

In [ ]:
!tree -d 'patients/TestMamaDer001'

Leemos los datos de un paciente

In [ ]:
pat = Patient('patients/TestMamaDer001')

En este caso el paciente solo contiene un estudio

In [ ]:
len(pat.studies)

Un conjunto de estructuras

In [ ]:
!ls patients/TestMamaDer001/20230116_095704/RTSTRUCT/CT/20230116_095704/

Y cuatro distribucioes de dosis

In [ ]:
!ls patients/TestMamaDer001/20230116_095704/RTDOSE/CT/20230116_095704/

Las cuatro distribuciones de dosis corresponden a cuatro planes, tres de fraccionamiento convencional (2 Gy/fr) y uno hipofraccionado (> 2 Gy/fr):
- Fraccionamiento convencional
    + Fase 1: Tratamiento uniforme de la prescripción sobre PTV mama dcha. ID: FraccStdF1
    + Fase 2: Tratamiento uniforme de la prescripción sobre PTV boost mama dcha. ID: FraccStdF2
    + Suma: Dosis Fase 1 + Dose Fase 2. ID: FraccStdF3
- Tratamiento hipofraccionado
    + Tratamiento simultáneo de los dos volúmenes objetivo (PTV mama dcha y PTV boost mama dcha) de manera que se llegue a las dosis totales de prescripción de cada uno de ellos simultáneamente. ID: MamaDerSIB 

In [ ]:
!ls patients/TestMamaDer001/20230116_095704/RTPLAN/CT/20230116_095704/

Creamos `plandict` variable de tipo diccionario que relaciona el nombre del plan con su UID

In [ ]:
plans = glob('patients/TestMamaDer001/20230116_095704/RTPLAN/CT/20230116_095704/*.dcm')
plandict = {}
for plan in plans:
    plandcm = dcm.read_file(plan)
    plandict[plandcm.RTPlanLabel] = plandcm.SOPInstanceUID

`plandict` permite obtener el UID de un plan a partir de su nombre. Por ejemplo el UID del plan suma (ID: FraccStdF3)

In [ ]:
plandict['FraccStdF3']

Creamos una variable sobre el estudio del paciente, y visualizamos el contenido del estudio. Observemos que `scikit-rt` ha leído correctamente el contenido y ha identificado una imagen, un conjunto de estructuras, cuatro planes y cuatro distribuciones de dosis.

In [ ]:
stdy = pat.studies[0]
stdy

Asignamos el conjunto de estructuras a una variable para facilitar su manejo. Fijémonos que estamos machacando la variable que habíamos definido previamente para mostrar el conjunto de estructuras, pero que en realidad estamos accediendo por dos cáminos diferentes a los mismos datos

In [ ]:
ss = stdy.ct_structure_sets[0]

### Distribución de dosis

Definimos una función que nos permita seleccionar la dosis por el nombre del plan

In [ ]:
def getrtdose(planname):
    for ct_dose in stdy.ct_doses:
        if ct_dose.plan.path.split('/')[-1] == 'RP.'  + plandict[planname] + '.dcm':
            return ct_dose 

Por ejemplo para recuperar la dosis del plan **FraccStdF1**

In [ ]:
getrtdose('FraccStdF1')

Definimos una función que nos permita visualizar la dosis correspondiente a un plan concreto

In [ ]:
@wdg.interact(plan=['FraccStdF1', 'FraccStdF2', 'FraccStdF3', 'MamaDerSIB'], z=(-50, 280), Dmin=(0, 65), Dmax=(1, 66))
def DosePlanViewer(plan, z=150, Dmin=0, Dmax=66):
    fig, ax = plt.subplots(figsize=(10,5))
    ss.plot(ax=ax, show=False, pos=z)
    dose = getrtdose(plan)
    dose.plot(pos=z, colorbar=1, ax=ax, intensity=[Dmin, Dmax], show=False)
    plt.show()

En la figura anterior es fácilmente observable, más aún si se hace zoom sobre la imagen, que el tamaño de pixel de las imágenes de CT es inferior al de la imagen de la distribución de dosis. Los valores reales de este estudio son un tamño de pixel de 1 mm en las imágenes de CT, mientras que en el planificador de radioterapia se empleó una matriz de cálculo con un tamaño de voxel de 2.5 mm. Estos valores son valores típicos empleados en la planificación, un compromiso entre precisión y esfuerzo de cálculo. Los planificadores de radioterapia no suelen mostrar la distribución de dosis en una vista como la anterior sino una más *suave*. Para ello los planificadores interpolan la distribución calculada. Esta interpolación puede realizarse sin falsear la información de la distribución de dosis porque esta no contiene frecuencias altas en la variación espacial, la propia naturaleza de la deposición de dosis impide la formación de bordes abruptos en la distribución de dosis.

`scikit-rt` proporciona herramientas para realizar la interpolación. Interpolamos todas las distribuciones de dosis (el proceso lleva tiempo, seamos pacientes). 

*La interpolación es exigente en cuanto a los recursos que necesita. Es posible que el kernel que corre en la nube se quede sin memoria y el proceso no se complete.*

In [ ]:
[ct_dose.resample(order=2) for ct_dose in stdy.ct_doses];

Si volvemos a representar la dosis ahora varía de forma tan suave como la imagen de CT

In [ ]:
@wdg.interact(plan=['FraccStdF1', 'FraccStdF2', 'FraccStdF3', 'MamaDerSIB'], z=(-50, 280), Dmin=(0, 65), Dmax=(1, 66))
def DosePlanViewer(plan, z=150, Dmin=0, Dmax=66):
    fig, ax = plt.subplots(figsize=(10,5))
    ss.plot(ax=ax, show=False, pos=z)
    dose = getrtdose(plan)
    dose.plot(pos=z, colorbar=1, ax=ax, intensity=[Dmin, Dmax], show=False)
    plt.show()

En la figura anterior podemos seleccionar los diferentes planes, cambiar el plano visualizado y el rango de dosis que se muestra para entender cómo es la distribución de dosis en cada caso.